In [4]:
import numpy as np
import pandas as pd
import os
import shutil
import cv2
from tqdm.notebook import tqdm

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext
from torchtext.datasets import Multi30k
#from torchtext.legacy.data import Field, BucketIterator, Iterator
import spacy


In [18]:
from torchtext.data  import Field, TabularDataset, BucketIterator, Iterator

In [5]:
from spacy import *

In [12]:
import spacy.cli 
spacy.cli.download("en_core_web_md")
spacy.cli.download("de_core_news_sm")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')


In [14]:
spacy_ger = spacy.load('de_core_news_sm')
spacy_eng= spacy.load('en_core_web_md')

In [33]:
def tokenizer_ger(text):
    return [tok.text for tok in spacy_ger.tokenizer(text)]
def tokenizer_eng(text):
    return [tok.text for tok in spacy_eng.tokenizer(text)]

In [34]:
german = Field(tokenize=tokenizer_ger, lower=True,
                init_token='<sos>', eos_token='<eos>')

english = Field(tokenize=tokenizer_eng, lower=True,
                init_token='<sos>', eos_token='<eos>')

In [35]:
train_data, validation_data, test_data = Multi30k.splits(exts=('.de','.en'),
                                                        fields=(german, english))

In [22]:
german.build_vocab(train_data, max_size=10000, min_freq=2)
english.build_vocab(train_data, max_size=10000, min_freq=2)

In [28]:
len(german.vocab)

7853

# New Section

In [20]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding.from_pretrained(ger_emb)
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=1, bidirectional=True)

    def forward(self, input_seq):
        embedded = self.embedding(input_seq)
        output, (hidden, cell) = self.lstm(embedded)
        return output, hidden, cell

# Create an instance of the Encoder model with the pre-trained embedding matrix
encoder = Encoder(input_size=10, hidden_size=128)

# Example usage of the Encoder model
input_seq = ger_sent[0] # example input sequence of shape (batch_size, sequence_length)
encoder_output, encoder_hidden, encoder_cell = encoder(input_seq)

In [21]:
encoder_hidden.shape

torch.Size([2, 128])

In [22]:
(creat_embed((eng_sent[0][0]))).unsqueeze(1).shape

torch.Size([10, 1])

In [23]:
class Decoder(nn.Module):
    def __init__(self, embedding_size, hidden_size, output_size, num_layers):
        super(Decoder, self).__init__()

        # Define the LSTM cell
        self.lstm = nn.LSTM(embedding_size, hidden_size, num_layers, batch_first=True)
        self.embedding = nn.Embedding.from_pretrained(eng_emb)
        # Define the output layer
        self.output_layer = nn.Linear(hidden_size, output_size)
    def forward(self, input_seq, hidden,cell):
        # Pass the input sequence and hidden state through the LSTM
        input_seq
        embedded = self.embedding(input_seq)
        output, (hidden, cell) = self.lstm(embedded.unsqueeze(0), (hidden, cell))

        # Pass the LSTM output through the output layer to get the output
        pred = self.output_layer(output)
        pred.squeeze(0)
        return pred,hidden,cell

decoder = Decoder(embedding_size=10, hidden_size=128, output_size=len(eng_vocab), num_layers=2)
input_seq = eng_sent[0][0] # replace with your input
 # use the final hidden state from the encoder
decoder_output, decoder_hidden,decoder_cell = decoder(input_seq, encoder_hidden,encoder_cell)

In [38]:
len(eng_vocab)

109333

In [36]:
decoder_output.argmax(1)

tensor([92069])

In [24]:
decoder_output.shape,len(eng_vocab)

(torch.Size([1, 109333]), 109333)

In [25]:
torch.zeros(2, 3, 5)[0]

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [26]:
torch.randint(0, 1000, (10, 1)).shape

torch.Size([10, 1])

In [27]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.in_embedding = nn.Embedding.from_pretrained(ger_emb)

    def forward(self, input_seq, target_seq):
        batch_size = 100
        target_len = 100
        target_vocab_size = len(eng_vocab)
        outputs = torch.zeros(target_len, batch_size, target_vocab_size)
        encoder_output, encoder_hidden, encoder_cell = self.encoder(input_seq)
        decoder_input = target_seq[0]
        for t in range(1, target_len):
            output, hidden, cell = self.decoder(decoder_input, encoder_hidden, encoder_cell)
            outputs[t] = output
            decoder_input = target_seq[t]
        return outputs

In [28]:
eng_sent.shape,ger_sent.shape

(torch.Size([602141, 100]), torch.Size([1331946, 100]))

In [29]:
num_epochs= 50
learning_rate = 0.001
batch_size = 64

In [30]:
encoder = Encoder(input_size=10, hidden_size=128)
decoder = Decoder(embedding_size=10, hidden_size=128, output_size=len(eng_vocab), num_layers=2)
model=Seq2Seq(encoder,decoder)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 
loss_fn = nn.CrossEntropyLoss()

In [31]:
encoder = Encoder(input_size=10, hidden_size=128)
decoder = Decoder(embedding_size=10, hidden_size=128, output_size=len(eng_vocab), num_layers=2)
model=Seq2Seq(encoder,decoder)

out=model(ger_sent[0],eng_sent[0])

In [ ]:
loss_fn(out,eng_sent[0])

In [ ]:
model.train()

# Driving training loop
c=0
for epoch in tqdm(range(1,num_epochs+1)):
        
    total_epoch_loss=0
    # Iterate through train dataset
    for inp,trg in zip(ger_sent,eng_sent):
        # 1. forward pass the inputs through the model
        if(c>1):break
        else:c+=1
        output =model(inp,trg)                           
        # 2. compute the loss for this iteration
        #labels=labels.type(torch.float)
        loss = loss_fn(output,trg)                                 
        total_epoch_loss += loss.item()

        # 3. zero out the gradients and perform backpropagation through the network
        optimizer.zero_grad()
        loss.backward()

        # 4. update the parameters
        optimizer.step()

    print(f"Epoch: [{epoch}/{num_epochs}] Epoch Loss: {total_epoch_loss}")

  0%|          | 0/50 [00:00<?, ?it/s]